# Feature Extraction for RNN
### Importing necessary libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
sns.set()
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import files

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
auth.authenticate_user()
print('Authenticated')


Authenticated


### Query to obtain features

In [3]:
project_id='eicudata'

In [4]:
%%bigquery --project $project_id rawdf
select b.uniquepid, b.patienthealthsystemstayid,b.patientunitstayid,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight, eyes, motor, verbal, (eyes+motor+verbal) as GCSTotal, temperature,respiratoryrate,heartrate,glucose,(select min(labresult)
from `physionet-data.eicu_crd.lab`
where patientunitstayid = a.patientunitstayid and labname like '%O2 Sat%'
group by labname) as O2,fio2,ph,hospitaldischargestatus,unitdischargestatus,(unitdischargeoffset-hospitaladmitoffset)/60/24 as ICUstay
from `physionet-data.eicu_crd.apacheapsvar` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
order by uniquepid;

In [5]:
rawdf

,uniquepid,patienthealthsystemstayid,patientunitstayid,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
0,002-10009,193705,224606,Female,76,Caucasian,"GI perforation/rupture, surgery for",160.0,NaN,3,6,5,14,36.7,38.0,119,81.0,NaN,100.0,7.280,Alive,Alive,3.031250
1,002-10018,178200,204602,Female,29,Caucasian,"Cardiovascular medical, other",162.6,88.5,4,6,5,15,36.5,55.0,138,82.0,NaN,-1.0,-1.000,Alive,Alive,4.201389
2,002-10034,141169,157016,Female,23,Caucasian,"GI medical, other",162.6,63.5,4,6,5,15,36.4,45.0,115,82.0,NaN,-1.0,-1.000,Alive,Alive,5.210417
3,002-10050,183274,211144,Female,67,Caucasian,Aortic valve replacement (isolated),160.0,86.2,4,6,5,15,36.8,33.0,104,85.0,94.0,50.0,7.380,Alive,Alive,3.893056
4,002-10050,190893,221005,Female,68,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,4,6,5,15,36.4,43.0,87,37.0,NaN,32.0,7.340,Alive,Alive,2.803472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,035-9957,2741786,3351785,Male,74,Caucasian,Head only trauma,182.9,75.7,4,6,4,14,36.8,49.0,50,-1.0,NaN,-1.0,-1.000,Alive,Alive,1.790278
171173,035-9959,2731423,3340321,Male,44,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,4,6,5,15,36.7,45.0,102,220.0,NaN,-1.0,-1.000,Alive,Alive,2.689583
171174,035-996,2736458,3345874,Male,55,African American,Rhythm disturbance (conduction defect),190.5,165.9,4,6,5,15,32.2,37.0,90,254.0,58.0,-1.0,-1.000,Alive,Alive,1.968056
171175,035-9966,2742533,3352628,Male,60,African American,"Apnea-sleep; surgery for (i.e., UPPP - uvulopa...",170.1,120.2,4,6,5,15,36.3,37.0,114,102.0,98.0,50.0,7.363,Alive,Alive,1.481250


### We will consider patients that have had multiple hospitalizations -

In [6]:
rawdf = rawdf[rawdf.groupby('patienthealthsystemstayid').patienthealthsystemstayid.transform(len) > 1]
rawdf

,uniquepid,patienthealthsystemstayid,patientunitstayid,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
20,002-10148,172762,197620,Female,69,Caucasian,Thoracotomy for lung cancer,160.0,53.0,4,6,5,15,35.9,6.0,90,413.0,NaN,30.0,7.250,Alive,Alive,0.743750
21,002-10148,172762,197617,Female,69,Caucasian,"ARDS-adult respiratory distress syndrome, non-...",160.0,57.1,4,6,5,15,35.0,9.0,90,154.0,NaN,50.0,7.230,Alive,Alive,8.811806
22,002-10148,172762,197618,Female,69,Caucasian,,160.0,57.1,-1,-1,-1,-3,-1.0,36.0,166,-1.0,NaN,-1.0,-1.000,Alive,Alive,3.343750
87,002-10556,199471,232031,Female,68,Caucasian,"Hemorrhage/hematoma, intracranial",157.5,80.7,4,6,5,15,36.3,59.0,105,158.0,NaN,-1.0,-1.000,Alive,Alive,2.638194
88,002-10556,199471,232032,Female,68,Caucasian,"Neoplasm-cranial, surgery for (excluding trans...",157.5,86.5,4,6,5,15,36.4,59.0,104,159.0,NaN,-1.0,-1.000,Alive,Alive,11.461806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171158,035-9870,2731845,3340781,Male,79,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,92.1,4,6,5,15,36.0,43.0,76,165.0,92.0,100.0,7.397,Alive,Alive,2.554167
171159,035-9870,2731845,3340779,Male,79,Caucasian,"Complications of previous open-heart surgery, ...",182.9,92.1,4,6,5,15,36.6,36.0,77,99.0,NaN,-1.0,-1.000,Alive,Alive,15.231250
171160,035-9870,2731845,3340780,Male,79,Caucasian,"Complications of previous open-heart surgery, ...",182.9,92.1,4,6,5,15,36.0,32.0,76,128.0,NaN,-1.0,-1.000,Alive,Alive,11.479167
171164,035-9891,2727434,3335903,Male,62,Caucasian,"Infarction, acute myocardial (MI)",167.6,65.1,4,6,5,15,36.7,14.0,150,112.0,90.0,-1.0,-1.000,Alive,Alive,2.759722


### Query to obtain some vital features

In [7]:
%%bigquery --project $project_id vital
select patientunitstayid, avg(systemicsystolic) as SystolicBP, avg(systemicdiastolic) as DiastolicBP, avg((2*systemicdiastolic + systemicsystolic)/3) as MAP
from `physionet-data.eicu_crd.vitalperiodic` 
where systemicsystolic & systemicdiastolic is not null
group by patientunitstayid
order by patientunitstayid

In [8]:
vital

,patientunitstayid,SystolicBP,DiastolicBP,MAP
0,141168,92.707317,51.000000,64.902439
1,141194,92.828488,50.636628,64.700581
2,141227,123.072581,78.064516,93.067204
3,141233,103.366634,55.650501,71.555879
4,141244,132.334862,57.155963,82.215596
...,...,...,...,...
46655,3353194,115.167622,61.166189,79.166667
46656,3353197,122.685241,49.463950,73.871047
46657,3353199,151.571674,41.611159,78.264664
46658,3353216,134.452465,77.519366,96.497066


### Merging all the features together

In [9]:
newrawdf = pd.merge(left=rawdf,right=vital,how='left')

In [10]:
newrawdf

,uniquepid,patienthealthsystemstayid,patientunitstayid,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,SystolicBP,DiastolicBP,MAP
0,002-10148,172762,197620,Female,69,Caucasian,Thoracotomy for lung cancer,160.0,53.0,4,6,5,15,35.9,6.0,90,413.0,NaN,30.0,7.250,Alive,Alive,0.743750,133.806604,41.797170,72.466981
1,002-10148,172762,197617,Female,69,Caucasian,"ARDS-adult respiratory distress syndrome, non-...",160.0,57.1,4,6,5,15,35.0,9.0,90,154.0,NaN,50.0,7.230,Alive,Alive,8.811806,128.037583,42.271186,70.859985
2,002-10148,172762,197618,Female,69,Caucasian,,160.0,57.1,-1,-1,-1,-3,-1.0,36.0,166,-1.0,NaN,-1.0,-1.000,Alive,Alive,3.343750,NaN,NaN,NaN
3,002-10556,199471,232031,Female,68,Caucasian,"Hemorrhage/hematoma, intracranial",157.5,80.7,4,6,5,15,36.3,59.0,105,158.0,NaN,-1.0,-1.000,Alive,Alive,2.638194,NaN,NaN,NaN
4,002-10556,199471,232032,Female,68,Caucasian,"Neoplasm-cranial, surgery for (excluding trans...",157.5,86.5,4,6,5,15,36.4,59.0,104,159.0,NaN,-1.0,-1.000,Alive,Alive,11.461806,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20075,035-9870,2731845,3340781,Male,79,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,92.1,4,6,5,15,36.0,43.0,76,165.0,92.0,100.0,7.397,Alive,Alive,2.554167,130.370242,61.205882,84.260669
20076,035-9870,2731845,3340779,Male,79,Caucasian,"Complications of previous open-heart surgery, ...",182.9,92.1,4,6,5,15,36.6,36.0,77,99.0,NaN,-1.0,-1.000,Alive,Alive,15.231250,NaN,NaN,NaN
20077,035-9870,2731845,3340780,Male,79,Caucasian,"Complications of previous open-heart surgery, ...",182.9,92.1,4,6,5,15,36.0,32.0,76,128.0,NaN,-1.0,-1.000,Alive,Alive,11.479167,108.935252,47.913669,68.254197
20078,035-9891,2727434,3335903,Male,62,Caucasian,"Infarction, acute myocardial (MI)",167.6,65.1,4,6,5,15,36.7,14.0,150,112.0,90.0,-1.0,-1.000,Alive,Alive,2.759722,NaN,NaN,NaN


### Checking for null values - 

In [11]:
newrawdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20080 entries, 0 to 20079
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   uniquepid                  20080 non-null  object 
 1   patienthealthsystemstayid  20080 non-null  int64  
 2   patientunitstayid          20080 non-null  int64  
 3   gender                     20080 non-null  object 
 4   age                        20080 non-null  object 
 5   ethnicity                  20080 non-null  object 
 6   apacheadmissiondx          20080 non-null  object 
 7   admissionheight            19987 non-null  float64
 8   admissionweight            18208 non-null  float64
 9   eyes                       20080 non-null  int64  
 10  motor                      20080 non-null  int64  
 11  verbal                     20080 non-null  int64  
 12  GCSTotal                   20080 non-null  int64  
 13  temperature                20080 non-null  flo

In [12]:
newrawdf.describe(include='all')

,uniquepid,patienthealthsystemstayid,patientunitstayid,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,SystolicBP,DiastolicBP,MAP
count,20080,2.008000e+04,2.008000e+04,20080,20080,20080,20080,19987.000000,18208.000000,20080.000000,20080.000000,20080.000000,20080.000000,20080.000000,20080.000000,20080.000000,20080.000000,9195.000000,20080.000000,20080.000000,20080,20080,20080.000000,5573.000000,5573.000000,5573.000000
unique,9196,NaN,NaN,4,76,7,354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,NaN,NaN,NaN,NaN
top,033-20426,NaN,NaN,Male,68,Caucasian,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,Alive,NaN,NaN,NaN,NaN
freq,10,NaN,NaN,11264,591,15326,1296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16808,19142,NaN,NaN,NaN,NaN
mean,NaN,1.331593e+06,1.673705e+06,NaN,NaN,NaN,NaN,169.527979,84.166430,3.108068,4.953137,3.465090,11.526295,32.344118,25.022649,98.076942,133.974104,90.617227,16.017276,1.265722,NaN,NaN,9.077183,121.508414,59.639779,80.262657
std,NaN,8.352836e+05,1.023435e+06,NaN,NaN,NaN,NaN,12.872213,27.125394,1.486119,2.100267,2.009484,5.320385,11.760448,15.759845,38.273893,99.008031,11.345977,31.148698,3.718256,NaN,NaN,11.304213,21.672877,14.679725,14.734310
min,NaN,1.290200e+05,1.412880e+05,NaN,NaN,NaN,NaN,1.600000,0.000000,-1.000000,-1.000000,-1.000000,-3.000000,-1.000000,-1.000000,-1.000000,-1.000000,-6.000000,-1.000000,-1.000000,NaN,NaN,-0.124306,-76.000000,-76.000000,-76.000000
25%,NaN,6.187980e+05,8.137948e+05,NaN,NaN,NaN,NaN,162.500000,66.000000,3.000000,5.000000,1.000000,10.000000,36.000000,10.000000,85.000000,82.000000,89.000000,-1.000000,-1.000000,NaN,NaN,2.290972,109.716377,52.073030,73.129819
50%,NaN,1.214896e+06,1.573048e+06,NaN,NaN,NaN,NaN,170.200000,80.000000,4.000000,6.000000,5.000000,14.000000,36.400000,28.000000,105.000000,115.000000,94.000000,-1.000000,-1.000000,NaN,NaN,5.706250,120.760000,58.798077,79.414477
75%,NaN,2.194127e+06,2.728451e+06,NaN,NaN,NaN,NaN,177.800000,97.300000,4.000000,6.000000,5.000000,15.000000,36.700000,36.000000,122.000000,185.000000,97.000000,30.000000,7.230000,NaN,NaN,11.805729,133.101291,65.864198,86.532832


### Imputing height and weight missing values 

In [13]:
newrawdf['admissionheight']=newrawdf.admissionheight.fillna(newrawdf.admissionheight.median())
newrawdf['admissionweight']=newrawdf.admissionweight.fillna(newrawdf.admissionweight.median())

### Converting age column to numeric datatype

In [14]:
newrawdf['age'].replace({'> 89': '90'}, inplace=True)
newrawdf['age']=newrawdf['age'].apply(pd.to_numeric)

In [15]:
newrawdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20080 entries, 0 to 20079
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   uniquepid                  20080 non-null  object 
 1   patienthealthsystemstayid  20080 non-null  int64  
 2   patientunitstayid          20080 non-null  int64  
 3   gender                     20080 non-null  object 
 4   age                        20078 non-null  float64
 5   ethnicity                  20080 non-null  object 
 6   apacheadmissiondx          20080 non-null  object 
 7   admissionheight            20080 non-null  float64
 8   admissionweight            20080 non-null  float64
 9   eyes                       20080 non-null  int64  
 10  motor                      20080 non-null  int64  
 11  verbal                     20080 non-null  int64  
 12  GCSTotal                   20080 non-null  int64  
 13  temperature                20080 non-null  flo

### Removing missing values for gender,ethnicity and hospital discharge status

In [16]:
missing = newrawdf[(newrawdf['gender'] == '') |(newrawdf['gender'] == 'Other')|(newrawdf['gender'] == 'Unknown')| (newrawdf['ethnicity'] == '')|(newrawdf['hospitaldischargestatus']=='')].index
newrawdf = newrawdf.drop(index=missing)
newrawdf = newrawdf.dropna()

In [17]:
newrawdf.isnull().sum()

uniquepid                    0
patienthealthsystemstayid    0
patientunitstayid            0
gender                       0
age                          0
ethnicity                    0
apacheadmissiondx            0
admissionheight              0
admissionweight              0
eyes                         0
motor                        0
verbal                       0
GCSTotal                     0
temperature                  0
respiratoryrate              0
heartrate                    0
glucose                      0
O2                           0
fio2                         0
ph                           0
hospitaldischargestatus      0
unitdischargestatus          0
ICUstay                      0
SystolicBP                   0
DiastolicBP                  0
MAP                          0
dtype: int64

In [18]:
newrawdf

,uniquepid,patienthealthsystemstayid,patientunitstayid,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,SystolicBP,DiastolicBP,MAP
10,002-10669,143252,159737,Male,60.0,Caucasian,CABG with aortic valve replacement,180.3,132.4,4,6,5,15,36.2,31.0,60,154.0,93.0,80.0,7.340,Alive,Alive,2.296528,110.166052,52.642066,71.816728
22,002-11177,138012,152898,Male,62.0,Caucasian,"Sepsis, pulmonary",167.6,151.4,4,6,5,15,38.5,22.0,90,125.0,97.0,60.0,7.310,Alive,Alive,9.627083,117.895753,56.379923,76.885199
26,002-11414,173515,198584,Female,34.0,Hispanic,"Arrest, respiratory (without cardiac arrest)",162.5,80.0,4,6,5,15,36.2,48.0,92,176.0,77.0,60.0,7.310,Expired,Alive,17.127778,100.685864,46.647906,64.660558
37,002-12088,163687,186042,Female,51.0,Caucasian,Coma/change in level of consciousness (for hep...,162.6,47.9,2,1,1,4,32.2,13.0,152,494.0,94.0,21.0,7.470,Alive,Alive,4.171528,95.789424,55.399433,68.862764
42,002-12226,189698,219467,Male,76.0,Caucasian,Thoracotomy for thoracic/respiratory infection,157.5,74.5,4,6,5,15,35.9,14.0,62,84.0,100.0,40.0,7.320,Alive,Alive,11.145833,100.328597,43.000000,62.109532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20067,035-9614,2738054,3347602,Male,58.0,Caucasian,"GI vascular ischemia, surgery for (resection)",170.1,61.9,4,6,4,14,37.1,30.0,124,106.0,89.0,-1.0,-1.000,Alive,Alive,12.600000,125.336449,54.439252,78.071651
20069,035-97,2732269,3341250,Female,76.0,Caucasian,CABG redo with valve repair/replacement,160.0,48.7,4,6,5,15,36.9,10.0,81,60.0,95.0,100.0,7.443,Alive,Alive,9.873611,105.912442,55.179724,72.090630
20073,035-9858,2724676,3332836,Male,54.0,Caucasian,Thoracotomy for thoracic/respiratory infection,167.6,80.6,4,6,5,15,36.0,30.0,58,95.0,99.0,44.0,7.356,Alive,Alive,5.476389,109.068966,61.807882,77.561576
20075,035-9870,2731845,3340781,Male,79.0,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,92.1,4,6,5,15,36.0,43.0,76,165.0,92.0,100.0,7.397,Alive,Alive,2.554167,130.370242,61.205882,84.260669


In [20]:
#ewrawdf['apacheadmissiondx'].unique()